In [97]:
from itertools import cycle, islice, repeat, product
from collections import Counter, defaultdict
from dataclasses import dataclass
from typing import Tuple

In [108]:
def wrap(n, lo=1, hi=10):
    return ((n - lo) % hi) + lo

@dataclass(frozen=True)
class Game:
    positions: Tuple[int] = (0, 0)
    scores: Tuple[int] = (0, 0)
    player: int = 0
        
    @staticmethod
    def starting_from(positions):
        return Game(positions)
        
    def move(self, steps):
        positions = list(self.positions)
        scores = list(self.scores)
        player = self.player
        positions[player] = wrap(positions[player] + steps)
        scores[player] += positions[player]
        return Game(tuple(positions), tuple(scores), self.next_player())
    
    def has_winner(self, score):
        return any(s >= score for s in self.scores)
    
    def next_player(self):
        return (self.player + 1) & 1

def deterministic_die(sides, rolls=3):
    die = cycle(range(1, sides+1))
    while True:
        yield sum(islice(die, rolls))
        
def play(die, positions=(1, 1), target=1000):
    rolls = 0
    game = Game.starting_from(positions)
    while not game.has_winner(target):
        game = game.move(next(die))
        rolls += 3
    return game.scores[game.player] * rolls


def dirac_die_rolls(sides, rolls=3):
    return Counter(sum(xs) for xs in
                   product(*(range(1, sides+1) for _ in range(rolls))))

def play_dirac(positions=(1, 1), target=21):
    dirac_rolls = dirac_die_rolls(3, 3)
    games = Counter([Game.starting_from(positions)])
    wins = [0, 0]
    while games:
        next_games = Counter()
        for steps, step_count in dirac_rolls.items():
            for game, game_count in games.items():
                game = game.move(steps)
                count = step_count * game_count
                if game.has_winner(target):
                    wins[game.next_player()] += count
                else:
                    next_games[game] += count
        games = next_games
    return wins

In [113]:
die = deterministic_die(100)
puzzle_start = (2, 10)
print('[p1] Test match result:', play(die, puzzle_start))
print('[p2] Dirac match result:', max(play_dirac(puzzle_start)))

[p1] Test match result: 571032
[p2] Dirac match result: 49975322685009
